Step 1 : Installing Dependecy

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install pandas sentence-transformers faiss-cpu gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.1 MB/s eta 0:00:00


In [3]:
from unsloth import FastModel
from unsloth.chat_templates import get_chat_template
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
import datasets
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


Step 2: Import of Dataset

In [4]:
# we will be use TruthfulQA as sample. Change to your dataset
ds = load_dataset("truthfulqa/truthful_qa", "generation")
df=ds['validation'].to_pandas()

README.md: 0.00B [00:00, ?B/s]

generation/validation-00000-of-00001.par(…):   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Step 3: Defining Functions

In [10]:

# Setup Encoding Model
def encode_model():
  model = SentenceTransformer("all-MiniLM-L6-v2")
  return model

# Setup LLm Model
def chat_model():

  model, tokenizer = FastModel.from_pretrained(
  model_name = "unsloth/gemma-3-4b-it", # change to model you like (based on huggingface model)
  max_seq_length = 2048, # Choose any for long context!
  load_in_4bit = True,  # 4 bit quantization to reduce memory
  load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
  full_finetuning = False, # [NEW!] We have full finetuning now!
  # token = "hf_...", # use one if using gated models
  )
  tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
  )
  return model,tokenizer


#Chat template for RAG
def generate_prompt(context_json,query):
  con_complete="Based only on the json formatted text given,"+query +". Give only answer: "+context_json
  return con_complete

# Creation of Vector Database (Using FAISS)
def vector_db():
  df['combined'] = df['question']+": " + df['best_answer'] # based on header of the file
  embeddings = model_encode.encode(df['combined'].tolist(), convert_to_numpy=True)
  dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(embeddings)
  return index

#  Vector Search query
def vector_search(que):
  query = que
  query_vec = model_encode.encode([query], convert_to_numpy=True)
  df['combined'] = df['question']+": " + df['best_answer'] # based on header of the file
  context = ""
  distances, relevant_indices = index.search(query_vec, k=1) # change to how many context. LLM can handle decently large Json based context
  for i in relevant_indices[0]:
    context += "\n"+"{'question': "+df.loc[i, 'question']+", 'best_answer': "+df.loc[i, 'best_answer']+"}" # manually Jsonize the data. // Propely way is to used df.to_json but it take some step thus use what easier
  return context

# Main System Function
def build_system2(user_question,history=[]):
  context=""
  context=vector_search(user_question)
  #print(context)
  prompt = generate_prompt(context, user_question)
  messages = [{
      "role": "user",
      "content": [{
      "type" : "text",
      "text" : prompt,
      }]
  }]
  text = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt = True, # Must add for generation
  )
  outputs = model_llm.generate(
      **tokenizer([text], return_tensors = "pt").to("cuda"),
      max_new_tokens = 256, # Increase for longer outputs!
      # Recommended Gemma-3 settings!
      temperature = 1.0, top_p = 0.95, top_k = 64,
      #skip_special_tokens = True
  )

  raw=tokenizer.batch_decode(outputs)[0]
  index_start= raw.index("<start_of_turn>model")+ len("<start_of_turn>model")
  index_end= raw.index("<end_of_turn>",index_start)
  response=raw[index_start:index_end]
  res=str(response)
  history.append((user_question, res))
  context=""
  return history,history


Step 4: Initiazation of Vector Database and LLM model

In [6]:
model_encode=encode_model()
index=vector_db()
model_llm,tokenizer=chat_model()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Step 5: Inferencing using Gradio UI

In [11]:
import gradio as gr

def clear(msg):
  msg2 = gr.Textbox(value="",placeholder="Ask a question", container=True)
  return msg2


css_template="""

    .back{
        background-image: url('https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Large_scale_structure_of_dark_matter.tif/lossy-page1-640px-Large_scale_structure_of_dark_matter.tif.jpg');

    }
"""
def demo():
    with gr.Blocks(theme=gr.themes.Ocean(primary_hue="blue", secondary_hue="blue"),css=css_template,elem_classes="back") as demo:
        with gr.Column(elem_classes="back"):
          gr.HTML("""
          <center style="color:white;"><h1 style="color:white;">Simple RAG System</h1>
          This is a prototype application for Simple RAG Systtem. <br>The application information based on the dataset given.
          <center>
          """)
          state = gr.State([])

          with gr.Row():
              with gr.Column():
                  chatbot = gr.Chatbot()
                  with gr.Row():
                      msg = gr.Textbox(placeholder="Ask a question", container=True)
                  with gr.Row():
                      submit_btn = gr.Button("Submit")

          submit_btn.click(build_system2, \
            inputs=[msg,state], \
            outputs=[chatbot,state], \
            queue=True)

          submit_btn.click(clear, \
            inputs=[msg], \
            outputs=[msg], \
            queue=True)




    demo.queue().launch(debug=True)

if __name__ == "__main__":
    demo()

/tmp/ipython-input-851952378.py:27: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d35fce2ea1bac51d4d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



{'question': Where did fortune cookies originate?, 'best_answer': The precise origin of fortune cookies is unclear}
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d35fce2ea1bac51d4d.gradio.live
